In [66]:
import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [67]:
import os

proxy = 'http://proxy-k.ecl.ntt.co.jp:8080'

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [68]:
TEST = False

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
L.seed_everything(random_seed)

Seed set to 1


1

In [69]:
from lightning.pytorch.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS


class DataModule(L.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
    
    def setup(self, stage=None):
        self.train_set = torchvision.datasets.MNIST(
            '/files/', train=True, download=True,
            transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.1307,), (0.3081,))
            ]))
        
        self.test_set = torchvision.datasets.MNIST(
            '/files/', train=False, download=True,
            transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.1307,), (0.3081,))
            ]))
        
    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return torch.utils.data.DataLoader(
            self.train_set, batch_size=batch_size_train, shuffle=True)
    
    def val_dataloader(self) -> EVAL_DATALOADERS:
        return torch.utils.data.DataLoader(
            self.test_set, batch_size=batch_size_test, shuffle=False)

    def test_dataloader(self) -> EVAL_DATALOADERS:
        return torch.utils.data.DataLoader(
            self.test_set, batch_size=batch_size_test, shuffle=False)

In [70]:
if TEST:
    data_module = DataModule()
    data_module.setup()

    print(data_module.train_set[0])

In [71]:
if TEST:
    image, y = data_module.train_set[0]
    random_output = torch.rand(1,10)

    print("image.shape", image.shape)
    plt.imshow(image[0])

    print("y", type(y), y)

In [72]:
if TEST:
    # NllLoss vs CrossEntLoss
    target = torch.tensor([y])
    print("random_output.shape", random_output.shape)
    print("target.shape", target.shape)

    output = F.log_softmax(random_output)
    print("output.shape", output.shape)

    loss = F.nll_loss(output, target)
    print("loss.shape", loss.shape)
    print("loss", loss)

    loss = F.cross_entropy(output, target)
    print("loss.shape", loss.shape)
    print("loss", loss)


In [73]:
if TEST:
    target = torch.tensor([y])
    output = F.log_softmax(random_output)

    print(target, output)
    print(torch.argmax(output, dim=1) == target)
    

In [74]:
from typing import Any
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler

class Net(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.rand(10, 1, 28, 28)

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
    def configure_optimizers(self) -> OptimizerLRScheduler:
        return optim.SGD(self.parameters(), lr=learning_rate,
                      momentum=momentum)
    
    def training_step(self, batch, batch_idx) -> STEP_OUTPUT:
        x, y = batch
        output = self.forward(x)
        loss = F.nll_loss(output, y)

        self.log("train_loss", loss)
        return loss
    
    def validation_step(self,  batch, batch_idx) -> STEP_OUTPUT:
        x, y = batch
        output = self.forward(x)
        loss = F.nll_loss(output, y)
        self.log("val_loss", loss)

        acc = (torch.argmax(output, dim=1) == y).sum()/len(y)
        self.log("val_acc", acc)

    def test_step(self,  batch, batch_idx) -> STEP_OUTPUT:
        x, y = batch
        output = self.forward(x)
        loss = F.nll_loss(output, y)
        self.log("test_loss", loss)

        acc = (torch.argmax(output, dim=1) == y).sum()/len(y)
        self.log("test_acc", acc)


In [75]:
net = Net()
data_module = DataModule() 

In [76]:
trainer = L.Trainer(max_epochs=n_epochs)
trainer.fit(model=net, datamodule=data_module)
trainer.test(model=net, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\1001002996\AppData\Local\Temp\ipykernel_13964\1767351907.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)

  | Name       | Type      | Params | In sizes         | Out sizes       
-------------------------------------------------------------------------------
0 | conv1      | Conv2d    | 260    | [10, 1, 28, 28]  | [10, 10, 24, 24]
1 | conv2      | Conv2d    | 5.0 K  | [10, 10, 12, 12] | [10, 20, 8, 8]  
2 | conv2_drop | Dropout2d | 0      | [10, 20, 8, 8]   | [10, 20, 8, 8]  
3 | fc1        | Linear    | 16.1 K | [10, 320]        | [10, 50]        
4 | fc2        | Linear    | 510    | [10, 50]         | [10, 10]        
-------------------------------------------------------------------------------
21.8 K    Trainable params
0         Non-trainable params
21.8 K    Total params
0.087     Total estimated model params size (MB)

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<?, ?it/s]

c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 938/938 [00:12<00:00, 77.59it/s, v_num=2]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 10.33it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9704999923706055
        test_loss           0.09313544631004333
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.09313544631004333, 'test_acc': 0.9704999923706055}]